<a href="https://colab.research.google.com/github/klordo/nlp_homeworks/blob/hw1/hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install corus

In [ ]:
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.1/lenta-ru-news.csv.bz2

In [ ]:
!pip install clean-text

In [ ]:
!pip install pymorphy2

In [ ]:
!python3 -m spacy download ru_core_news_sm

In [ ]:
!pip install wordninja

In [ ]:
from corus import load_lenta2
from cleantext import replace_emails, replace_numbers, replace_urls
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
import pymorphy2
import spacy
import wordninja
import re

In [8]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
path = 'lenta-ru-news.csv.bz2'
records = load_lenta2(path)

In [10]:
dataset = [next(records).text for i in range(1000)]
dataset[0]

'Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили. Вылазки гарнизона Перемышля остаются безуспешными. При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новые партии пленных, орудия и прочая материальная часть. На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии. «Русский инвалид», 16 сентября 1914 года.'

In [11]:
dataset_clean = dataset.copy()
dataset_clean = [replace_emails(text, ' email ') for text in dataset_clean]
dataset_clean = [replace_numbers(text, ' number ') for text in dataset_clean]
dataset_clean = [replace_urls(text, ' url ') for text in dataset_clean]

In [12]:
symbols = set()
for text in dataset_clean:
    for char in text:
        if not char.isalpha() and not char.isdigit() and char not in ' .,!?':
            symbols.add(char)
symbols

{'"',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 '-',
 '/',
 ':',
 ';',
 '=',
 '\xa0',
 '«',
 '»',
 '–',
 '—',
 '’',
 '…',
 '№'}

In [13]:
pattern = r'[^\w\s\d.,!?]+'
dataset_clean = [re.sub(pattern, '', text) for text in dataset_clean]

In [14]:
dataset_clean[0]

'Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра  number  сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили. Вылазки гарнизона Перемышля остаются безуспешными. При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новые партии пленных, орудия и прочая материальная часть. На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии. Русский инвалид,  number  сентября  number  года.'

nltk и pymorphy2

In [15]:
dataset_tokenized = [[word_tokenize(sent, language='russian') for sent in sent_tokenize(text, language='russian')] for text in dataset_clean]
dataset_tokenized[0][:2]

[['Бои',
  'у',
  'Сопоцкина',
  'и',
  'Друскеник',
  'закончились',
  'отступлением',
  'германцев',
  '.'],
 ['Неприятель',
  ',',
  'приблизившись',
  'с',
  'севера',
  'к',
  'Осовцу',
  'начал',
  'артиллерийскую',
  'борьбу',
  'с',
  'крепостью',
  '.']]

In [16]:
morph = pymorphy2.MorphAnalyzer()

In [17]:
dataset_lemmatized = [[[morph.parse(word)[0].normal_form for word in sent] for sent in text] for text in dataset_tokenized]

In [18]:
dataset_lemmatized[0][:2]

[['бой',
  'у',
  'сопоцкина',
  'и',
  'друскеник',
  'закончиться',
  'отступление',
  'германец',
  '.'],
 ['неприятель',
  ',',
  'приблизиться',
  'с',
  'север',
  'к',
  'осовца',
  'начать',
  'артиллерийский',
  'борьба',
  'с',
  'крепость',
  '.']]

Вариант с помощью spacy для токенизации, лемматизации и удаления стоп слов.

In [19]:
nlp = spacy.load("ru_core_news_sm")

In [20]:
dataset_doc = [[token.lemma_ for token in nlp(doc) if not token.is_stop] for doc in dataset_clean]

In [21]:
dataset_doc[0][:10]

['бой',
 'сопоцкина',
 'друскеник',
 'закончиться',
 'отступление',
 'германец',
 '.',
 'неприятель',
 ',',
 'приблизиться']

In [22]:
bad_tokens = set()
for i, text in enumerate(dataset_doc):
    for word in text:
        if re.search(r'[^a-zA-Zа-яА-ЯёЁ]', word) and word not in '.,!?':
            bad_tokens.add(word)

list(bad_tokens)[:20]

['95полетов',
 'огарева,6',
 'в.матвиенко',
 'в20',
 '.zip',
 'границу,436',
 'а.',
 '12сентября',
 'ил76',
 '.com',
 'составляет61',
 'января1999',
 'suppl.doc',
 'сб.',
 '.237пунктов',
 'мюнстера25',
 '92миллиардов',
 'в.у.',
 '...',
 'п.бородин']

Решил убрать плохие токены. Либо детально очищать текст, либо потерять часть информации. Я выбрал второе. Таких токенов по сравнению с размером датасета в целом не так много.

In [23]:
dataset_doc = [[word for word in text if word not in bad_tokens] for text in dataset_doc]

По проблеме слов без пробелов.

Использовал словарь с семинара, но многие слова некорректно делились, видимо из-за того, что многих специфичных и не очень слов в словаре не было.